# Evaluation of our approach

In this notebook, we include the test routine we used to compare our method, with the intention to help anyway trying to reproduce the obtained results. 

This notebook assumes that the datasets were cleaned using the [preprocessing notebook](./data_preprocessing.ipynb).
The results of our run were saved to [losses.json](./.results/losses.json).

If any bugs were to be caught while reading this code or while reproducing the results, please let us know on the project's [GitHub page](https://github.com/rompoggi/MCTS_ClassifierChain) via a [Pull Request](https://github.com/rompoggi/MCTS_ClassifierChain/pulls) or the [Discussions](https://github.com/rompoggi/MCTS_ClassifierChain/discussions) channel.

In [1]:
ds_names = [
    "2-EMOT",
    "3-SCENE",
    "4-FLAGS",
    "5-FOODTRUCK",
    "6-YEAST",
    "7-BIRDS",
    "8-GENBASE",
    # "9-MEDC",          # Removed as too long to test on
    # "10-ENRON",        # Removed as too long to test on
    # "11-MEDIAMILL",    # Removed as too long to test on
    ]

import pandas as pd
def get_dataset(ds: str):
    path_X = f"./data/datasets/{ds}_X.csv"
    path_y = f"./data/datasets/{ds}_y.csv"
    X = pd.read_csv(path_X)
    y = pd.read_csv(path_y)
    return X, y

In [4]:
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold
import numpy as np
import math
from sklearn.metrics import hamming_loss, zero_one_loss
from datetime import datetime 

from mcts_inference.pcc import PCC
from mcts_inference.mcts import MCTS
from mcts_inference.mcc import MCC
from mcts_inference.policy import UCB, EpsGreedy, Thompson_Sampling
from mcts_inference.constraints import Constraint
from mcts_inference.mcts_config import MCTSConfig

def losses(X, y, chain, algo, config, loss_fns=[hamming_loss]):
    if algo is None:
        y_pred = chain.predict(X)
    elif config is not None:
        y_pred = np.abs(algo(X, chain, config))
    else:
        raise ValueError("Config cannot be None if algo is not None")

    return [fn(y, y_pred) for fn in loss_fns]

def loss_algos(ds, k=5, n_repeats = 1, random_state=0, loss_fns=[hamming_loss, zero_one_loss], loss_dict={}):
    d_time = 20.
    n_iter = 5
    
    loss_dict[ds] = {}
    loss_dict[ds]["PCC"] = []
    loss_dict[ds]["CC"] = []
    loss_dict[ds]["MCC"] = []
    loss_dict[ds]["MCTS UCB(2)"] = []
    loss_dict[ds]["MCTS EpsGreedy(0.2)"] = []
    loss_dict[ds]["MCTS EpsGreedy(0.5)"] = []
    loss_dict[ds]["MCTS Thompson_Sampling(1,1)"] = []
    loss_dict[ds]["MCT1S UCB(2)"] = []
    loss_dict[ds]["MCT1S EpsGreedy(0.2)"] = []

    X, y = get_dataset(ds)
    
    chain = ClassifierChain(SVC(max_iter=10_000, gamma="auto", probability=True, random_state=random_state))

    rkf = RepeatedKFold(n_splits=k, n_repeats=n_repeats, random_state=random_state)

    print(f" - {n_iter=}, {d_time=}, k={k}, n_repeats={n_repeats}")
    i = 1
    for train_idx, test_idx in rkf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        print(f"[{' '*(int(math.log(n_repeats*k,10))-int(math.log(i,10)))}{i}/{n_repeats * k} ", end="")
        if (not (y_train.nunique() > 1).all()):  # Check if there are multiple classes in the training set
            print(f"missing ", end="")
            # continue
            dummy_row = pd.DataFrame({col: [0 if y_train[col].iloc[0] == 1 else 1] for col in y_train.columns}, index=[y_train.index.max() + 1])
            y_train = pd.concat([y_train, dummy_row])
            dummy_x_row = pd.DataFrame(np.zeros((1, len(X_train.columns))), columns=X_train.columns, index=[X_train.index.max() + 1])
            X_train = pd.concat([X_train, dummy_x_row])


        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        chain = chain.fit(X_train, y_train)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)


        chain = chain.fit(X_train, y_train)
        loss = losses(X_test, y_test, chain, algo=None, config=None, loss_fns=loss_fns)
        loss_dict[ds]["CC"].append(loss)   
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], constraint=Constraint(time=True, d_time=d_time))
        loss = losses(X_test, y_test, chain, algo=PCC, config=config, loss_fns=loss_fns)
        loss_dict[ds]["PCC"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=UCB(2), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_test, y_test, chain, MCC, config, loss_fns=loss_fns)
        loss_dict[ds]["MCC"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=UCB(2), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_test, y_test, chain, MCTS, config, loss_fns=loss_fns)
        loss_dict[ds]["MCTS UCB(2)"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=EpsGreedy(0.2), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_test, y_test, chain, MCTS, config, loss_fns=loss_fns)
        loss_dict[ds]["MCTS EpsGreedy(0.2)"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=EpsGreedy(0.5), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_test, y_test, chain, MCTS, config, loss_fns=loss_fns)
        loss_dict[ds]["MCTS EpsGreedy(0.5)"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=Thompson_Sampling(1,1), constraint=Constraint(max_iter=True, n_iter=n_iter))
        loss = losses(X_test, y_test, chain, MCTS, config, loss_fns=loss_fns)
        loss_dict[ds]["MCTS Thompson_Sampling(1,1)"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=UCB(2), constraint=Constraint(max_iter=True, n_iter=n_iter), step_once=False)
        loss = losses(X_test, y_test, chain, MCTS, config, loss_fns=loss_fns)
        loss_dict[ds]["MCT1S UCB(2)"].append(loss)
        print(f". ", end="")

        config = MCTSConfig(n_classes=y.shape[1], selection_policy=EpsGreedy(0.2), constraint=Constraint(max_iter=True, n_iter=n_iter), step_once=False)
        loss = losses(X_test, y_test, chain, MCTS, config, loss_fns=loss_fns)
        loss_dict[ds]["MCT1S EpsGreedy(0.2)"].append(loss)
        print(f". {datetime.now().strftime('%H:%M:%S')}]")

        i+=1

    for key in loss_dict[ds].keys():
        loss_dict[ds][key] = list(np.mean(loss_dict[ds][key], axis=0))

    return loss_dict[ds]

In [5]:
from sklearn.metrics import hamming_loss, zero_one_loss
from datetime import datetime
import json

losses_dict = dict()
print("- Hamming Loss & Zero One Loss-")
for ds in ds_names[:7]:
    print(">"*80)
    print(f"> Dataset: {ds} - {datetime.now().strftime('%H:%M:%S')}", end="")
    try:
        L = loss_algos(ds, k=10, n_repeats=2, random_state=0, loss_fns=[hamming_loss, zero_one_loss], loss_dict=losses_dict)
        print("_"*32)
        print(f"Brute Force (PCC)           : {L['PCC']}, Score: {[1- x for x in L['PCC']]}")
        print(f"Classifier Chain (CC)       : {L['CC']}, Score: {[1- x for x in L['CC']]}")
        print(f"Monte Carlo CC (MCC)        : {L['MCC']}, Score: {[1- x for x in L['MCC']]}")
        print(f"MCTS UCB(2)                 : {L['MCTS UCB(2)']}, Score: {[1- x for x in L['MCTS UCB(2)']]}")
        print(f"MCTS EpsGreedy(0.2)         : {L['MCTS EpsGreedy(0.2)']}, Score: {[1-x for x in L['MCTS EpsGreedy(0.2)']]}")
        print(f"MCTS EpsGreedy(0.5)         : {L['MCTS EpsGreedy(0.5)']}, Score: {[1-x for x in L['MCTS EpsGreedy(0.5)']]}")
        print(f"MCTS Thompson_Sampling(1,1) : {L['MCTS Thompson_Sampling(1,1)']}, Score: {[1-x for x in L['MCTS Thompson_Sampling(1,1)']]}")
        print(f"MCT1S UCB(2)                : {L['MCT1S UCB(2)']}, Score: {[1- x for x in L['MCT1S UCB(2)']]}")
        print(f"MCT1S EpsGreedy(0.2)        : {L['MCT1S EpsGreedy(0.2)']}, Score: {[1- x for x in L['MCT1S EpsGreedy(0.2)']]}")
        print("_"*32,"\n")

        with open('./data/.results/temp_losses.json', 'a') as outfile:  # Save results to file
            L["datetime"] = datetime.now().strftime('%H:%M:%S')
            json.dump(L, outfile)
            del L["datetime"]  # Remove this key as we do not save it if the whole test goes through

    except Exception as e:
        print(f"Error in {ds}: {e}")
        losses_dict[ds] = None

losses_dict["datetime"] = datetime.now().strftime('%H:%M:%S')
with open('./data/.results/losses.json', 'a') as outfile:  # Save all results
    json.dump(losses_dict, outfile)

- Hamming Loss & Zero One Loss-
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
> Dataset: 2-EMOT - 16:23:08 - n_iter=5, d_time=20.0, k=10, n_repeats=2
[ 1/20 . . . . . . . . . 16:23:22]
[ 2/20 . 

KeyboardInterrupt: 